In [ ]:
# import libraries and packages 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.stats import uniform, randint
from sklearn import model_selection, linear_model, metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedShuffleSplit, KFold
from sklearn.metrics import auc, accuracy_score, confusion_matrix, roc_auc_score, classification_report

import xgboost as xgb
import seaborn as sns

In [ ]:
df = pd.read_csv("credit_risk_dataset.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [ ]:
copy_df = df.copy()

In [ ]:
df.shape

(32581, 12)

In [ ]:
"""
Person age: tuổi hiện tại
Person income: thu nhập hàng năm tại thời điểm cho vay (income/year)
Person home ownership: loại hình sỡ hữu nhà (nơi đang sống hiện tại)
Person employment length: thời gian làm việc (tính theo năm)
Loan intent: mục đích vay
Loan amount: số tiền vay / khoản vay
Loan interest rate: tỉ lệ lãi suất cho vay
loan percent income: tiền vay / thu nhập.
cb person default on file: trước đó đã từng vỡ nợ hay chưa.
cb person cred hist length: khoản vay trước đó cách khoản vay hiện tại bao nhiêu năm
"""

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,32581.0,27.734600,6.348078,20.00,23.00,26.00,30.00,144.00
person_income,32581.0,66074.848470,61983.119168,4000.00,38500.00,55000.00,79200.00,6000000.00
person_emp_length,31686.0,4.789686,4.142630,0.00,2.00,4.00,7.00,123.00
loan_amnt,32581.0,9589.371106,6322.086646,500.00,5000.00,8000.00,12200.00,35000.00
loan_int_rate,29465.0,11.011695,3.240459,5.42,7.90,10.99,13.47,23.22
loan_status,32581.0,0.218164,0.413006,0.00,0.00,0.00,0.00,1.00
loan_percent_income,32581.0,0.170203,0.106782,0.00,0.09,0.15,0.23,0.83
cb_person_cred_hist_length,32581.0,5.804211,4.055001,2.00,3.00,4.00,8.00,30.00


Từ bảng thống kê trên ta thấy một số điểm vô lý

-độ tuổi lớn nhất là 144 => outlier

-thời gian làm việc 123 => outlier

In [ ]:
# Phần trăm giá trị NaN trên từng thuộc tính.
NaN_percent = df.isnull().sum()
NaN_percent

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Hai giá trị tồn tại null/NaN là person_emp_length vaf loan_int_rate

Dựa vào biểu đồ ta có thể thấy độ tuổi cũng những người trong dataset nằm phần lớn trong khoảng từ 20 đến 60 tuổi, 1 phần nhỏ từ 60->80    
Loại bỏ những outlier về độ có độ tuổi > 90 và có số năm làm việc <20.   
Loại bỏ những outlier có > 120 năm kinh nghiệm nhưng độ tuổi chỉ ở 20

In [ ]:
df = df[df["person_age"] < 90]
df = df[df["person_emp_length"] < 120]

Biểu đồ cho ta thấy được loan_grade được đánh giá tương quan với giá trị loan interest rate.

In [ ]:
Split data: original
train - validation - test => constrain 
kfold validation = 35000 => 5 tap nho => A B C D E

Logistic Regression, Decision Tree (Random Forest), SVM, Neural Network, Kmean.

GridSearch, RandomizedSearch

Metrics:
Accuracy
Confusion Matrix
Precision/Recall
F1 Score / Average Precision / AUC 

In [ ]:
df[df["loan_status"] == 1].shape[0], df[df["loan_status"] == 0].shape[0]

(7108, 25473)

In [ ]:
df.select_dtypes("object")

,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file
0,RENT,PERSONAL,D,Y
1,OWN,EDUCATION,B,N
2,MORTGAGE,MEDICAL,C,N
3,RENT,MEDICAL,C,N
4,RENT,MEDICAL,C,Y
...,...,...,...,...
32576,MORTGAGE,PERSONAL,C,N
32577,MORTGAGE,PERSONAL,A,N
32578,RENT,HOMEIMPROVEMENT,B,N
32579,MORTGAGE,PERSONAL,B,N


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df["person_home_ownership"])
le.classes_
le.transform(df["person_home_ownership"])

array([3, 2, 0, ..., 3, 0, 3])

In [ ]:
df["person_home_ownership"]

0            RENT
1             OWN
2        MORTGAGE
3            RENT
4            RENT
           ...   
32576    MORTGAGE
32577    MORTGAGE
32578        RENT
32579    MORTGAGE
32580        RENT
Name: person_home_ownership, Length: 32581, dtype: object

In [ ]:
(7108 / 35000, 25473/35000)

(0.20308571428571429, 0.7278)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
type(df.select_dtypes("object"))

pandas.core.frame.DataFrame

In [ ]:
enc = OrdinalEncoder()
enc.fit(df.select_dtypes("object"))

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [ ]:
enc.categories_

[array(['MORTGAGE', 'OTHER', 'OWN', 'RENT'], dtype=object),
 array(['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL',
        'PERSONAL', 'VENTURE'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype=object),
 array(['N', 'Y'], dtype=object)]

In [ ]:
enc.transform(df.select_dtypes("object")).shape

(31678, 4)

In [ ]:
class TransformToNumerical(BaseEstimator, TransfomerMixin):
    def __init__(self, object_cols):
        

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()